In [21]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.dpi"] = 70
import seaborn as sns

%load_ext autoreload
%autoreload 2

[Неплохой разбор, есть определения переменных](http://rstudio-pubs-static.s3.amazonaws.com/290261_676d9bb194ae4c9882f599e7c0a808f2.html)

In [38]:
df = pd.read_csv('data/rejected_2007_to_2018Q4.csv')

[autoreload of utils failed: Traceback (most recent call last):
  File "/Users/user/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/user/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/user/opt/anaconda3/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/user/opt/anaconda3/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/user/credi

In [55]:
from utils import load_csv_compressed

accepted = load_csv_compressed('data/accepted_2007_to_2018Q4.csv')

In [3]:
accepted['label'] = accepted['loan_status'].replace(('Fully Paid', 
                                                      'Charged Off',
                                                      'Does not meet the credit policy. Status:Fully Paid',
                                                      'Does not meet the credit policy. Status:Charged Off',
                                                     'Default'
                                                    ),
                                                               (0, 1, 0, 1, 1))

In [4]:
accepted['label'] = accepted['label'].replace('Default', 1)

In [5]:
accepted.drop(index=accepted[~accepted.label.isin([0, 1])].index, inplace=True)

In [6]:
accepted.drop(columns=accepted.columns[((accepted.isna().sum() / accepted.shape[0] * 100) > 10).values], inplace=True)

In [7]:
numeric = accepted.select_dtypes(include=np.number).columns
categorical = accepted.select_dtypes(exclude=np.number).columns

In [8]:
corr = accepted[numeric].corr()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
loan_amnt,1.000000,0.999545,0.998214,0.141432,0.953435,0.311851,0.032548,-0.002912,0.101525,0.101524,...,-0.022199,-0.020764,0.089646,0.021674,-0.094254,0.007577,0.335855,0.276394,0.372893,0.200108
funded_amnt,0.999545,1.000000,0.998777,0.141519,0.954105,0.311742,0.032952,-0.002716,0.100681,0.100680,...,-0.022201,-0.020766,0.089646,0.021673,-0.094056,0.007712,0.335854,0.276394,0.372901,0.200105
funded_amnt_inv,0.998214,0.998777,1.000000,0.141589,0.952714,0.311208,0.034054,-0.002410,0.099887,0.099886,...,-0.022215,-0.020782,0.089660,0.021622,-0.093681,0.008014,0.335883,0.276401,0.372934,0.200109
int_rate,0.141432,0.141519,0.141589,1.000000,0.153703,-0.072114,0.146776,0.048301,-0.405484,-0.405481,...,0.026725,0.208280,-0.063515,0.251162,0.058847,0.011762,-0.119004,0.003636,-0.229684,-0.004205
installment,0.953435,0.954105,0.952714,0.153703,1.000000,0.302610,0.036243,0.006540,0.051547,0.051547,...,-0.016670,-0.000408,0.072058,0.041112,-0.086347,0.014536,0.303059,0.261041,0.340721,0.185375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tax_liens,0.007577,0.007712,0.008014,0.011762,0.014536,0.037787,-0.022950,0.011228,-0.057701,-0.057701,...,0.002163,0.016856,-0.033913,-0.018514,0.035863,1.000000,-0.001579,-0.002010,-0.024981,0.005971
tot_hi_cred_lim,0.335855,0.335854,0.335883,-0.119004,0.303059,0.411355,0.014401,0.058160,0.196957,0.196958,...,0.005857,0.088273,0.025668,-0.018067,-0.107868,-0.001579,1.000000,0.507682,0.397174,0.400466
total_bal_ex_mort,0.276394,0.276394,0.276401,0.003636,0.261041,0.316955,0.226223,0.031896,0.032828,0.032825,...,0.015696,0.119801,0.063874,0.061256,-0.078053,-0.002010,0.507682,1.000000,0.285927,0.872655
total_bc_limit,0.372893,0.372901,0.372934,-0.229684,0.340721,0.270169,0.026529,-0.075717,0.375048,0.375048,...,-0.058701,0.042170,0.215088,-0.215060,-0.154062,-0.024981,0.397174,0.285927,1.000000,0.117485


In [9]:
# plt.figure(figsize=(50, 50))

# mask = np.triu(np.ones_like(corr, dtype=bool))

# sns.heatmap(corr, vmin=-1, vmax=1, annot=True, mask=mask)

In [10]:
accepted.drop(columns='policy_code', inplace=True)

In [11]:
def calcDrop(res):
    # All variables with correlation > cutoff
    all_corr_vars = list(set(res['v1'].tolist() + res['v2'].tolist()))
    
    # All unique variables in drop column
    poss_drop = list(set(res['drop'].tolist()))

    # Keep any variable not in drop column
    keep = list(set(all_corr_vars).difference(set(poss_drop)))
     
    # Drop any variables in same row as a keep variable
    p = res[ res['v1'].isin(keep)  | res['v2'].isin(keep) ][['v1', 'v2']]
    q = list(set(p['v1'].tolist() + p['v2'].tolist()))
    drop = (list(set(q).difference(set(keep))))

    # Remove drop variables from possible drop 
    poss_drop = list(set(poss_drop).difference(set(drop)))
    
    # subset res dataframe to include possible drop pairs
    m = res[ res['v1'].isin(poss_drop)  | res['v2'].isin(poss_drop) ][['v1', 'v2','drop']]
        
    # remove rows that are decided (drop), take set and add to drops
    more_drop = set(list(m[~m['v1'].isin(drop) & ~m['v2'].isin(drop)]['drop']))
    for item in more_drop:
        drop.append(item)
         
    return drop
    

def corrX(df, cut = 0.9) :
    # Get correlation matrix and upper triagle
    corr_mtx = df.corr().abs()
    avg_corr = corr_mtx.mean(axis = 1)
    up = corr_mtx.where(np.triu(np.ones(corr_mtx.shape), k=1).astype(np.bool))
    
    dropcols = list()
    
    res = pd.DataFrame(columns=(['v1', 'v2', 'v1.target', 
                                 'v2.target','corr', 'drop' ]))
    
    for row in range(len(up)-1):
        col_idx = row + 1
        for col in range (col_idx, len(up)):
            if(corr_mtx.iloc[row, col] > cut):
                if(avg_corr.iloc[row] > avg_corr.iloc[col]): 
                    dropcols.append(row)
                    drop = corr_mtx.columns[row]
                else: 
                    dropcols.append(col)
                    drop = corr_mtx.columns[col]
                
                s = pd.Series([ corr_mtx.index[row],
                up.columns[col],
                avg_corr[row],
                avg_corr[col],
                up.iloc[row,col],
                drop],
                index = res.columns)
        
                res = res.append(s, ignore_index = True)
    
    dropcols_names = calcDrop(res)
    
    return dropcols_names

In [12]:
to_drop = corrX(accepted, cut = .6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


['out_prncp_inv',
 'total_rec_prncp',
 'mo_sin_rcnt_rev_tl_op',
 'acc_open_past_24mths',
 'revol_util',
 'total_rev_hi_lim',
 'bc_util',
 'delinq_2yrs',
 'num_rev_tl_bal_gt_0',
 'funded_amnt_inv',
 'num_rev_accts',
 'pub_rec',
 'num_bc_sats',
 'total_bc_limit',
 'num_op_rev_tl',
 'num_actv_rev_tl',
 'total_bal_ex_mort',
 'total_acc',
 'total_pymnt',
 'tot_hi_cred_lim',
 'recoveries',
 'fico_range_low',
 'loan_amnt',
 'tot_cur_bal',
 'last_fico_range_high',
 'funded_amnt',
 'acc_now_delinq',
 'total_pymnt_inv',
 'open_acc']

In [13]:
to_drop.remove('loan_amnt')

In [14]:
len(to_drop)

28

In [15]:
numeric = accepted.select_dtypes(include=np.number).columns
categorical = accepted.select_dtypes(exclude=np.number).columns

In [16]:
to_drop += ['installment', 'total_rec_int', 'last_pymnt_amnt', 'id']

In [17]:
# corr_new = accepted[numeric].drop(columns = to_drop).corr()

# plt.figure(figsize=(50, 50))

# mask = np.triu(np.ones_like(corr_new, dtype=bool))

# sns.heatmap(corr_new, vmin=-1, vmax=1, annot=True, mask=mask)

In [18]:
accepted['label'] = pd.to_numeric(accepted['label'])

In [19]:
to_drop_categorical = ['last_pymnt_d', 'last_credit_pull_d', 'url', 'issue_d', 'debt_settlement_flag', 'hardship_flag', 'zip_code', 'title', 'pymnt_plan']

In [20]:
accepted.drop(columns=to_drop + to_drop_categorical, inplace=True)

In [26]:
accepted.drop(columns=['id', 'last_pymnt_amnt'], inplace=True)

In [39]:
with pd.option_context('display.max_rows', 0): 
  count = accepted['emp_title'].value_counts().sort_values(ascending=False)
  print(count[count > 1000])

Teacher               21268
Manager               19472
Owner                 10302
Registered Nurse       8774
RN                     8522
Supervisor             8290
Driver                 7559
Sales                  7488
Project Manager        6381
Office Manager         5526
General Manager        5191
Director               5032
                      ...  
mechanic               1193
Secretary              1183
Manager                1118
Financial Analyst      1113
Superintendent         1110
Bartender              1088
Financial Advisor      1086
Physician              1082
Officer                1080
Pharmacist             1065
CNA                    1046
Accounting Manager     1010
Name: emp_title, Length: 78, dtype: int64


In [41]:
import yaml

print(yaml.dump(accepted.columns.values.tolist(), explicit_start=True, default_flow_style=False))

---
- loan_amnt
- term
- int_rate
- grade
- sub_grade
- emp_title
- emp_length
- home_ownership
- annual_inc
- verification_status
- purpose
- addr_state
- dti
- earliest_cr_line
- fico_range_high
- inq_last_6mths
- revol_bal
- initial_list_status
- out_prncp
- total_rec_late_fee
- collection_recovery_fee
- last_fico_range_low
- collections_12_mths_ex_med
- application_type
- tot_coll_amt
- avg_cur_bal
- bc_open_to_buy
- chargeoff_within_12_mths
- delinq_amnt
- mo_sin_old_il_acct
- mo_sin_old_rev_tl_op
- mo_sin_rcnt_tl
- mort_acc
- mths_since_recent_bc
- num_accts_ever_120_pd
- num_actv_bc_tl
- num_bc_tl
- num_il_tl
- num_sats
- num_tl_120dpd_2m
- num_tl_30dpd
- num_tl_90g_dpd_24m
- num_tl_op_past_12m
- pct_tl_nvr_dlq
- percent_bc_gt_75
- pub_rec_bankruptcies
- tax_liens
- total_il_high_credit_limit
- disbursement_method
- label



In [1]:
from pipeline import get_preprocessing_pipeline

In [2]:
pipeline = get_preprocessing_pipeline()
pipeline

Pipeline(memory='./cache',
         steps=[('ColumnSelector',
                 ColumnSelector(columns=['loan_amnt', 'loan_status', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti', 'earliest_cr_line', 'fico_range_high', 'inq_last_6mths', 'revol_bal', 'initial_list_status', 'out_prncp', 'tota...mths_since_recent_bc', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_bc_tl', 'num_il_tl', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'total_il_high_credit_limit', 'disbursement_method'])),
                ('WingsOfEvidence', WingsOfEvidence(verbose=True))])

In [3]:
import datatable as dt

In [4]:
accepted = dt.fread('data/archive/accepted.csv')

In [ ]:
pipeline.fit_transform(accepted)